<a href="https://colab.research.google.com/github/ArtoAnnila/GalaxyDynamiX/blob/main/GalaxyDynamiX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*

# **GalaxyDynamiX**

* Run cells to get figures.
* Edit code to change parameters and profiles.



In [ ]:
#@title MOUNT
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
#@title VELOCITY DISPERSIONS AND ROTATION CURVES

# GalaxyCurveX: Computes galaxy velocity dispersion and rotation curves
# including the acceleration effect of cosmic expansion.

import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gammaincinv
from scipy.optimize import minimize_scalar
from scipy.integrate import quad

path="/content/gdrive/MyDrive/ColabMaps/"

plt.rcParams['font.family'] = 'DejaVu Serif'

# ---------------- Constants ----------------
G = 6.67430e-11              # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                # Speed of light, m/s
M_sun = 1.98847e30           # Solar mass, kg
ly_in_m = 9.461e15           # Light-year in meters
pc_in_m = 3.0857e16          # Parsec in meters
t_universe = 13.8e9 * 365.25 * 24 * 3600  # Age of universe in seconds
a_R = c / t_universe         # Cosmic acceleration
rho = 1/(4*math.pi*G*t_universe**2)   # universal density kgm^-3
#rho = c**2*c*t_universe/(4*math.pi*G*(c*t_universe)**3)

# ---------------- Galaxy Parameters ----------------
M_total = 1e11 * M_sun       # Total galaxy mass
#M_total = 1e8 * M_sun        # Dwarf

# Hernquist (Elliptical Galaxy)
r_scale_ly = 10000 #2000
#r_scale_ly = 4000 #500             # Dwarf
r_scale_m = r_scale_ly * ly_in_m

# Sérsic (Spiral Galaxy)
R_e_ly_ser = 20000
#R_e_ly_ser = 1000 #500             # Dwarf
R_e_pc_ser = R_e_ly_ser * ly_in_m / pc_in_m
n_ser = 2.0
#n_ser = 1.0                  # Dwarf
bn = gammaincinv(2 * n_ser, 0.5)
mass_to_light_ratio_ser = 1.0

# ---------------- Sérsic Profile Setup ----------------
def find_I_e_ser(target_mass, R_e_pc, n, b_n, mass_to_light_ratio):
    def integrand(r_pc, I_e):
        return (2 * math.pi * r_pc *
                math.exp(-bn * ((r_pc / R_e_pc)**(1/n) - 1)) *
                mass_to_light_ratio * M_sun * I_e)

    def mass_function(I_e):
        result, _ = quad(integrand, 0, np.inf, args=(I_e,))
        return (result - target_mass)**2

    result = minimize_scalar(mass_function, bounds=(1e-3, 1e3), method='bounded')
    return result.x

I_e_ser = find_I_e_ser(M_total, R_e_pc_ser, n_ser, bn, mass_to_light_ratio_ser)

def I_Sersic(r_pc):
    return I_e_ser * math.exp(-bn * ((r_pc / R_e_pc_ser)**(1/n_ser) - 1))

def Sigma_ser(r_pc):
    return I_Sersic(r_pc) * mass_to_light_ratio_ser

def integrand_ser(r_pc):
    return 2 * math.pi * r_pc * Sigma_ser(r_pc) * M_sun

def M_Sersic(r_ly):
    r_pc = r_ly * ly_in_m / pc_in_m
    result, _ = quad(integrand_ser, 0, r_pc)
    return result

def ecv_Sersic(r_ly): # Newtonian estimate of circular velocity
    M_r = M_Sersic(r_ly)
    r_m = r_ly * ly_in_m
    return math.sqrt(G * M_r / r_m)

def ecv_Total(r_ly): # Estimate of circular velocity including universal acceleration
    M_r = M_Sersic(r_ly)
    r_m = r_ly * ly_in_m
    a_o = G * M_r / r_m**2
    rho_o = 3*M_r / (4 * math.pi * r_m**3)
    f = max(0, 1 - rho / rho_o)
    return (f*(a_o + a_R/(2*math.pi)) * G * M_r)**0.25

# ---------------- Hernquist Model ----------------
def M_enclosed(r_ly):
    r_m = r_ly * ly_in_m
    return M_total * (r_m / (r_m + r_scale_m))**2

def rho_hern(r_m):
    return (M_total * r_scale_m) / (2 * np.pi * r_m * (r_m + r_scale_m)**3)

def accel_Newton(r_m):
    a_o = G * M_enclosed(r_m) / r_m**2
    return a_o

def accel_total(r_m):
    a_o = G * M_enclosed(r_m) / r_m**2
    return a_o + a_R

def evd_Hernquist(r_ly,
                  r_max_factor=200.0,
                  n_grid=2000,
                  include_aR=True):
    scalar_input = np.isscalar(r_ly)
    r_ly_arr = np.atleast_1d(r_ly).astype(float)
    r_m_arr = r_ly_arr * ly_in_m
    r_min = np.maximum(1e-10 * r_scale_m, np.min(r_m_arr))
    r_max = r_scale_m * r_max_factor
    r_grid = np.logspace(np.log10(r_min), np.log10(r_max), n_grid)
    integrand = rho_hern(r_grid) * accel_Newton(r_grid)
    dr = np.diff(r_grid)
    segment = 0.5 * (integrand[1:] + integrand[:-1]) * dr
    cum = np.concatenate(([0.0], np.cumsum(segment)))
    total_integral = cum[-1]

    cum_at_r = np.interp(r_m_arr, r_grid, cum)
    integral_r_to_rmax = total_integral - cum_at_r

    rho_r = rho_hern(r_m_arr)
    f = max(0, 1 - rho / rho_hern(r_m_arr))

    sigma2 = integral_r_to_rmax / rho_r
    sigma2 = np.where(sigma2 > 0, sigma2, 0.0)
    sigma_s = np.sqrt(sigma2)
    sigma4 = integral_r_to_rmax*G*r_m_arr**2
    sigma_s = (f*sigma4)**0.25
    # analytic σ(0)
    sigma0 = np.sqrt(G * M_total / (12.0 * r_scale_m))
    small_mask = (r_m_arr < 1e-3 * r_scale_m)
    #sigma_s[small_mask] = np.minimum(sigma_s[small_mask], sigma0)
    if scalar_input:
        return float(sigma_s[0])
    return sigma_s

def evd_Total(r_ly,
                  r_max_factor=200.0,
                  n_grid=2000,
                  include_aR=True):
    scalar_input = np.isscalar(r_ly)
    r_ly_arr = np.atleast_1d(r_ly).astype(float)
    r_m_arr = r_ly_arr * ly_in_m
    r_min = np.maximum(1e-10 * r_scale_m, np.min(r_m_arr))
    r_max = r_scale_m * r_max_factor
    r_grid = np.logspace(np.log10(r_min), np.log10(r_max), n_grid)
    integrand = rho_hern(r_grid) * accel_total(r_grid)
    dr = np.diff(r_grid)
    segment = 0.5 * (integrand[1:] + integrand[:-1]) * dr
    cum = np.concatenate(([0.0], np.cumsum(segment)))
    total_integral = cum[-1]

    cum_at_r = np.interp(r_m_arr, r_grid, cum)
    integral_r_to_rmax = total_integral - cum_at_r

    rho_r = rho_hern(r_m_arr)
    f = max(0, 1 - rho / rho_hern(r_m_arr))
    sigma2 = integral_r_to_rmax / rho_r
    sigma2 = np.where(sigma2 > 0, sigma2, 0.0)
    sigma_s = np.sqrt(sigma2)
    sigma4 = integral_r_to_rmax*G*r_m_arr**2
    sigma_s = (f*sigma4)**0.25
    # analytic σ(0)
    sigma0 = np.sqrt(G * M_total / (12.0 * r_scale_m))
    small_mask = (r_m_arr < 1e-3 * r_scale_m)
    #sigma_s[small_mask] = np.minimum(sigma_s[small_mask], sigma0)
    if scalar_input:
        return float(sigma_s[0])
    return sigma_s


# ---------------- Compute Profiles ----------------
#radii_ly = np.logspace(2, 6, 100)  # 100 ly to 1,000,000 ly
radii_ly = np.logspace(2, 8, 100)  # 100 ly to 100,000,000 ly

vd_Hernquist = [evd_Hernquist(r) for r in radii_ly]
vd_Total = [evd_Total(r) for r in radii_ly]
cv_Sersic = [ecv_Sersic(r) for r in radii_ly]
cv_Total = [ecv_Total(r) for r in radii_ly]

vd_Hernquist_km_s = np.array(vd_Hernquist) / 1000
vd_Total_km_s = np.array(vd_Total) / 1000
cv_Sersic_km_s = np.array(cv_Sersic) / 1000
cv_Total_km_s = np.array(cv_Total) / 1000

def plot_velocity_profiles(radii, vd_total, vd_newtonian, cv_total, cv_newtonian,
                           xscale='linear', xlim=None, ylim=None, filename=None):  # Added ylim here
    fig, ax1 = plt.subplots(figsize=(8, 7))

    # Velocity dispersion
    ax1.plot(radii, vd_total, ':', lw=4, color='black', label="Total velocity dispersion")
    ax1.plot(radii, vd_newtonian, ':', lw=3, color='gray', label="Newtonian velocity dispersion")

    # Circular velocity
    ax1.plot(radii, cv_total, '-', lw=3, color='black', label="Total circular velocity")
    ax1.plot(radii, cv_newtonian, '-', lw=2, color='gray', label="Newtonian circular velocity")

    # Axis formatting
    ax1.set_xlabel("$r$ (light-years)", fontsize=20)
    ax1.set_ylabel("$\\sigma$ (km/s)", fontsize=20)
    ax1.set_xscale(xscale)
    if xlim:
        ax1.set_xlim(xlim)
    if ylim:
        ax1.set_ylim(ylim)  # Apply y-axis limits
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.grid(True)

    # Secondary y-axis
    ax2 = ax1.twinx()
    ax2.set_ylabel("$v$ (km/s)", fontsize=20)
    ax2.tick_params(axis='both', which='major', labelsize=16)
    ax2.set_ylim(ax1.get_ylim())  # Match the primary y-axis

    #ax1.legend(fontsize=12)
    plt.tight_layout()
    if filename:
        plt.savefig(path+filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as: {filename}")
    plt.show()

# Usage
plot_velocity_profiles(
    radii=radii_ly,
    vd_total=vd_Total_km_s,
    vd_newtonian=vd_Hernquist_km_s,
    cv_total=cv_Total_km_s,
    cv_newtonian=cv_Sersic_km_s,
    ylim=(0, 250),
#    xlim=(1e2, 1e7),
    xlim=(1e2, 1.5e6),
    xscale='log',
    filename="log_plot.jpg"
)

plot_velocity_profiles(
    radii=radii_ly,
    vd_total=vd_Total_km_s,
    vd_newtonian=vd_Hernquist_km_s,
    cv_total=cv_Total_km_s,
    cv_newtonian=cv_Sersic_km_s,
    xscale='linear',
    ylim=(0, 250),
    xlim=(0, 100000),
#    xlim=(0, 2000000),
#    xlim=(0, 2000000),
    filename="lin_plot.jpg"
)

# --- Density and Acceleration Functions ---


# Hernquist Density Profile
def rho_Hernquist(r_ly):
    r_m = r_ly * ly_in_m
    return (M_total * r_scale_m) / (2 * math.pi * r_m * (r_m + r_scale_m)**3)

# Sersic Density Profile
#def rho_Sersic_surface(r_ly):
def rho_Sersic(r_ly):
    r_pc = r_ly * ly_in_m / pc_in_m
    r_m = r_ly * ly_in_m
#    surface_density = Sigma_ser(r_pc) * M_sun  # kg/m^2
    surface_density = Sigma_ser(r_pc)  # kg/m^2
    return surface_density / (2 * r_m)  # Approximate 3D density assuming spherical symmetry

# Calculate b_n
def b_n(n):
    return 2 * n - 0.327

# Calculate p
def p_n(n):
    return 1.0 - 0.6097 / n + 0.05463 / n**2

# Approximate 3D Sersic density profile (Prugniel-Simien)
def rho_Sersic_3D(r_ly):
#def rho_Sersic(r_ly):
    r_m = r_ly * ly_in_m
    r_pc = r_m / pc_in_m
    bn = b_n(n_ser)
    pn = p_n(n_ser)
    #Re_m = R_e_pc_ser * pc_in_m
    Re_m = R_e_ly_ser * ly_in_m

    # rho0: approximate normalization
    rho0 = (M_total / (4 * math.pi * n_ser * (Re_m)**3))  # simple guess normalization, can adjust

    x = r_m / Re_m
    return rho0 * (x)**(-pn) * math.exp(-bn * (x)**(1/n_ser))

# Hernquist Acceleration Profile
def acc_Hernquist(r_ly):
    r_m = r_ly * ly_in_m
    M_enc = M_enclosed(r_ly)
    return G * M_enc / r_m**2

# Sersic Acceleration Profile
def acc_Sersic(r_ly):
    r_m = r_ly * ly_in_m
    M_enc = M_Sersic(r_ly)
    return G * M_enc / r_m**2

# --- Compute Density and Acceleration Profiles ---

rho_Hernquist_vals = [rho_Hernquist(r) for r in radii_ly]
rho_Sersic_vals = [rho_Sersic(r) for r in radii_ly]
acc_Hernquist_vals = [acc_Hernquist(r) for r in radii_ly]
acc_Sersic_vals = [acc_Sersic(r) for r in radii_ly]

# --- Plot Density and Acceleration Profiles ---

def plot_density_acceleration(radii, rho_hern, rho_sersic, acc_hern, acc_sersic,
                               xscale='log', xlim=None, ylim_rho=None, ylim_acc=None):
    fig, ax1 = plt.subplots(figsize=(8, 7))

    # Density (left axis)
    ax1.plot(radii, rho_hern, lw=2, color='blue', label="Hernquist density")
    ax1.plot(radii, rho_sersic, lw=2, color='green', label="Sérsic density")
    ax1.axhline(y=rho, color='black', lw=4, label="Universal density")

    ax1.set_xlabel("$r$ (light-years)", fontsize=20)
    ax1.set_ylabel("Density (kg/m³)", color='blue', fontsize=20)
    ax1.set_xscale(xscale)
    ax1.set_yscale('log')
    if xlim:
        ax1.set_xlim(xlim)
    if ylim_rho:
        ax1.set_ylim(ylim_rho)
    ax1.tick_params(axis='y', which='major', labelsize=16, colors='blue')
    ax1.tick_params(axis='x', which='major', labelsize=16, colors='black')
    ax1.grid(True)

    # Acceleration (right axis)
    ax2 = ax1.twinx()
    ax2.plot(radii, acc_hern, '--', lw=2, color='red', label="Hernquist acceleration")
    ax2.plot(radii, acc_sersic, '--', lw=2, color='orange', label="Sérsic acceleration")
    ax2.axhline(y=a_R, linestyle='--', lw=4, color='black', label="Universal acceleration")

    ax2.set_ylabel("Acceleration (m/s²)", color='red', fontsize=20)
    ax2.set_xscale(xscale)
    ax2.set_yscale('log')
    if ylim_acc:
        ax2.set_ylim(ylim_acc)
    ax2.tick_params(axis='both', which='major', labelsize=16, colors='red')

    # Combine legends
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, fontsize=12, loc='lower left')

    plt.tight_layout()
    plt.show()

# Usage
plot_density_acceleration(
    radii=radii_ly,
    rho_hern=rho_Hernquist_vals,
    rho_sersic=rho_Sersic_vals,
    acc_hern=acc_Hernquist_vals,
    acc_sersic=acc_Sersic_vals,
    xscale='log',
    ylim_rho=(1e-33, 1e-15),
    ylim_acc=(1e-15, 1e-6),
    xlim=(1e2, 1e8)
)

plot_density_acceleration(
    radii=radii_ly,
    rho_hern=rho_Hernquist_vals,
    rho_sersic=rho_Sersic_vals,
    acc_hern=acc_Hernquist_vals,
    acc_sersic=acc_Sersic_vals,
    xscale='linear',
    ylim_rho=(1e-33, 1e-15),
    ylim_acc=(1e-15, 1e-6),
    xlim=(0, 100000)
)


In [ ]:
#@title FLUX BALANCE BLOW-UP
# Plot overdensity falloff near flux balance

import numpy as np
import math
import matplotlib.pyplot as plt

# ---------------- Constants ----------------
G = 6.67430e-11              # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                # Speed of light, m/s
M_sun = 1.98847e30           # Solar mass, kg
ly_in_m = 9.461e15           # Light-year in meters
pc_in_m = 3.0857e16          # Parsec in meters
t_universe = 13.8e9 * 365.25 * 24 * 3600  # Age of universe in seconds
a_R = c / t_universe         # Cosmic acceleration
rho = 1/(4*math.pi*G*t_universe**2)   # universal density kgm^-3
#rho = c**2*c*t_universe/(4*math.pi*G*(c*t_universe)**3)

# ---------------- Galaxy Parameters ----------------
M_total = 1e11 * M_sun       # Total galaxy mass

# Compute characteristic radius
r_c_m = (M_total / rho)**(1/3)  # in meters
r_c_ly = r_c_m / ly_in_m  # convert meters to light-years

# Define radius range (slightly below and above r_c)
radii_ly = np.linspace(0.05*r_c_ly, 1.4*r_c_ly, 500)
radii_r_by_rc = radii_ly / r_c_ly  # dimensionless r/r_c

# --- Define Functions ---

# Exact cubic model
def delta_rho_exact(r_by_rc):
    return 1 - r_by_rc**3

# Logistic approximation
def delta_rho_logistic(r_by_rc):
    return 1 / (1 + r_by_rc**3)**2

# Cutoff power law approximation
def delta_rho_cutoff(r_by_rc, n=3):
    return 1 / (1 + r_by_rc**n)

# --- Evaluate ---
delta_exact = delta_rho_exact(radii_r_by_rc)
delta_logistic = delta_rho_logistic(radii_r_by_rc)
delta_cutoff = delta_rho_cutoff(radii_r_by_rc, n=3)

# --- Plotting ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10), sharex=False)

# Linear scale plot
ax1.plot(radii_r_by_rc, delta_exact, label=r'Exact: $1 - (r/r_{fb})^3$', lw=2)
ax1.plot(radii_r_by_rc, delta_logistic, '--', label=r'Logistic: $1/(1+(r/r_{fb})^3)^2$', lw=2)
ax1.plot(radii_r_by_rc, delta_cutoff, ':', label=r'Cutoff power law: $1/(1+(r/r_{fb})^3)$', lw=2)
ax1.axvline(1, color='gray',  lw=3, linestyle='--', label=r'$r=r_{fb}$')
ax1.axhline(0, color='gray', lw=3, linestyle='--', label=r'$\delta_\rho=0$')
ax1.set_xlabel(r'$r/r_{fb}$', fontsize=12)
ax1.set_ylabel(r'$\delta_\rho$', fontsize=12)
ax1.set_title('Overdensity $\delta_\\rho$ vs. $r/r_{fb}$', fontsize=14)
ax1.legend()
ax1.grid(True)
ax1.set_ylim(-1, 1.2)
ax1.set_xlim(0.1, 1.2)

# Log-x scale plot
ax2.plot(radii_r_by_rc, delta_exact, label=r'Exact: $1 - (r/r_{fb})^3$', lw=2)
ax2.plot(radii_r_by_rc, delta_logistic, '--', label=r'Logistic: $1/(1+(r/r_{fb})^3)^2$', lw=2)
ax2.plot(radii_r_by_rc, delta_cutoff, ':', label=r'Cutoff power law: $1/(1+(r/r_{fb})^3)$', lw=2)
ax2.axvline(1, color='gray', lw=3, linestyle='--', label=r'$r=r_{fb}$')
ax2.axhline(0, color='gray', lw=3, linestyle='--', label=r'$\delta_\rho=0$')
ax2.set_xscale('log')
ax2.set_xlabel(r'$r/r_{fb}$ (log scale)', fontsize=12)
ax2.set_ylabel(r'$\delta_\rho$', fontsize=12)
ax2.set_title('Overdensity $\delta_\\rho$ vs. $r/r_{fb}$', fontsize=14)
ax2.legend()
ax2.grid(True)
ax2.set_ylim(-1.1, 1.2)
ax2.set_xlim(0.1, 1.2)

plt.tight_layout()
plt.show()


In [ ]:
#@title DENSITIES ACROSS COSMOS
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator

plt.rcParams['font.family'] = 'DejaVu Serif'

# ---------------- Constants ----------------
G = 6.67430e-11                 # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                   # Speed of light, m/s
year = 365.25*24*3600 # 3.154e7 # seconds in a year
lightyear =  9.461e15           # Light-year in meters
age = 13800000000*365.25*24*3600

rho = np.logspace(-27, 19, 500) # Density range: from 1e-30 to 1e20 kg/m^3
rho_now = 6.28663585785629E-27
t = np.sqrt(1 / (4 * np.pi * G * rho)) / year  # time in years
r = np.sqrt(c**2 / (4 * np.pi * G * rho)) / lightyear  # distance in lightyears

# Characteristic densities (approximate values in kg/m^3)
objects = {
    'Stellar black hole': 1e18,           # ~10 M☉ within ~30 km (Schwarzschild radius)
    'Neutron star': 1e17,                 # Typical 1.4 M☉ in ~10 km radius
    'White dwarf': 1e9,                   # E.g., Sirius B (~1 M☉ in Earth-sized radius)
    'Sun': 1.41e3,                        # Mean solar density (reference star)
    'Red dwarf': 1e2,                     # Red dwarfs (e.g., Proxima Centauri)
    'Giant star': 1e-5,                   # E.g., Betelgeuse Much lower average density due to size
    'Globular cluster': 1e-17,            # E.g., Omega Centauri
    'Dwarf galaxy': 1e-20,                # E.g., Large Magellanic Cloud
    'Spiral galaxy': 1e-21,               # E.g., Milky Way average
    'Elliptical galaxy': 1e-22,           # E.g., M87
    'Ultra-diffuse galaxy': 1e-23,        # E.g., Dragonfly 44
    'Galaxy group': 1e-24,                # E.g., Local Group
    'Galaxy cluster': 1e-25,              # E.g., Virgo Cluster
    'Supercluster': 1e-26,                # E.g., Laniakea Supercluster
#    'Voids': 1e-27,                       #
}

# Density range: from 1e-30 to 1e20 kg/m^3
rho = np.logspace(-27, 19, 500)
rho_now = 6.28663585785629E-27

# Time and radius
t = np.sqrt(1 / (4 * np.pi * G * rho)) / year  # time in years
r = c*t / lightyear  # radius in lightyears

rho_points = np.array(list(objects.values()))
t_points = np.sqrt(1 / (4 * np.pi * G * rho_points)) / year

# ---------------- Plotting ----------------
fig, ax1 = plt.subplots(figsize=(8, 6))

# Left y-axis: time
ax1.loglog(rho, t, label=r"$t = \left(\frac{1}{4\pi G \rho}\right)^{1/2}$", linewidth=1, color='gray')
ax1.scatter(rho_points, t_points, color='white', edgecolor='black', linewidth=1, s=100, zorder=5)

# Annotations
for name, x, y in zip(objects.keys(), rho_points, t_points):
    ax1.annotate(name, (x, y), textcoords="offset points", xytext=(8,-5), ha='left', fontsize=7.5)

# Axis labels
ax1.set_xlabel(r"$\rho$ (kg/m³)", fontsize=16)
ax1.set_ylabel(r"$t$ (years)", fontsize=16)
ax1.tick_params(axis='both', labelsize=12)
ax1.set_xlim(1e-33, 1e19)
ax1.set_ylim(1e-15, 1e12)
ax1.invert_xaxis()  # <<< Reverse the x-axis
ax1.xaxis.set_major_locator(LogLocator(base=10.0, numticks=10))
ax1.yaxis.set_major_locator(LogLocator(base=10.0, numticks=10))
ax1.grid(True, which="both", ls="--", lw=0.5)

"""
# Right y-axis: radius
ax2 = ax1.twinx()
ax2.set_yscale('log')

# Synchronize r = ct with t in years
tmin, tmax = ax1.get_ylim()
ax2.set_ylim(c * tmin * year, c * tmax * year)
ax2.set_ylabel(r"$r = ct$ (m)", fontsize=16)
ax2.tick_params(axis='y', labelsize=12)
ax2.yaxis.set_major_locator(LogLocator(base=10.0, numticks=10))
"""
plt.tight_layout()
plt.savefig("plot.png", dpi=300, bbox_inches='tight')
plt.show()
from google.colab import files
files.download("plot.png")